In [2]:
# Import statements
import yfinance as yf
import pandas as pd
import os
import numpy as np
from datetime import date
from pathlib import Path
import pandas_ta as ta
from price_data_pull_yfinance import gethistoricalOHLC, saveHistStockData, loadHistDataFromDisk

from global_vars import dataPathToOHLC, pathToMasterDF, float16Cols, float32Cols

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
masterDf = pd.read_csv(pathToMasterDF);
masterDf.head()

,TICKER,FIRST_DATE_OHLC,LAST_DATE_OHLC,FILEPATH
0,MMM,1990-01-02,2021-03-19,C:\Users\Philipp\Google Drive\Code_for_bothCom...
1,AOS,1990-01-02,2021-03-19,C:\Users\Philipp\Google Drive\Code_for_bothCom...
2,ABT,1990-01-02,2021-03-19,C:\Users\Philipp\Google Drive\Code_for_bothCom...
3,ABBV,2013-01-02,2021-03-19,C:\Users\Philipp\Google Drive\Code_for_bothCom...
4,ABMD,1990-01-02,2021-03-19,C:\Users\Philipp\Google Drive\Code_for_bothCom...


In [14]:
aapl.tail(1)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
5336,2021-03-18,122.875,123.1875,120.3125,120.5,121229696.0,0.0,0.0


In [32]:
def loadHistDataFromDisk(
    ticker, database=None, pathToDF=pathToMasterDF, online_search=True
):
    """tries to load data from disk, if available.
    In case it does not work, it tries to pull the data from yahoo finance.
    Full docu to be written"""
    # TODO #4

    assert isinstance(
        ticker, str
    ), f"Ticker should be a string but is actually of type {type(ticker)}"

    if database is None:
        try:
            database = pd.read_csv(pathToMasterDF)
        except:
            print(
                f"could not open the database dataframe. Looking for a csv under the file location {pathToDF}"
            )

    if ticker in database["TICKER"].values:

        pathToFile = database.loc[database["TICKER"] == ticker, "FILEPATH"].values[0]

        tickerDF = pd.read_feather(pathToFile)
    else:
        print("Could not find the files you requested!")
        if online_search == True:
            print("Try pulling the data from yfinance!")
            try:
                tickerDF = gethistoricalOHLC(ticker)
            except:
                print("Could not load the data from yfinance!")
    tickerDF["Date"] = pd.to_datetime(tickerDF["Date"])
    return tickerDF

In [33]:
aapl = loadHistDataFromDisk('AAPL1');
aapl.tail(1)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
5336,2021-03-18,122.875,123.1875,120.3125,120.5,121229696.0,0.0,0.0


In [54]:
masterDf

,TICKER,FIRST_DATE_OHLC,LAST_DATE_OHLC,FILEPATH


In [42]:
dftest = aapl[['Date','Open', 'High', 'Low', 'Close', 'Volume']];
dftest.set_index(pd.DatetimeIndex(dftest['Date']), inplace = True);
dftest = dftest.drop('Date',axis = 1);
dftest.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2021-03-08,120.9375,121.0000,116.1875,116.3750,153918592.0
2021-03-09,119.0000,122.0625,118.8125,121.0625,129159600.0
2021-03-10,121.6875,122.1875,119.4375,120.0000,111760400.0
2021-03-11,122.5625,123.1875,121.2500,121.9375,102753600.0
2021-03-12,120.3750,121.1875,119.1875,121.0000,87963400.0


In [22]:
dftest.ta.log_return(cumalative=True,append=True);
dftest

,open,high,low,close,volume,LOGRET_1
date,,,,,,
1999-12-31,0.776367,0.791016,0.765137,0.790527,163811200.0,NaN
2000-01-03,0.806641,0.865234,0.782227,0.860840,535796800.0,0.085205
2000-01-04,0.832520,0.850586,0.778320,0.788086,512377600.0,-0.088257
2000-01-05,0.797852,0.850098,0.791992,0.799805,778321600.0,0.014771
2000-01-06,0.816406,0.822754,0.730469,0.730469,767972800.0,-0.090576
...,...,...,...,...,...,...
2021-03-08,120.937500,121.000000,116.187500,116.375000,153918592.0,-0.042969
2021-03-09,119.000000,122.062500,118.812500,121.062500,129159600.0,0.039062
2021-03-10,121.687500,122.187500,119.437500,120.000000,111760400.0,-0.007812


In [43]:
dftest.ta.strategy('Volume')

In [44]:
dftest.tail()

,open,high,low,close,volume,EFI_13,EOM_14_100000000,MFI_14,NVI_1,OBV,PVI_1,PVOL,PVR,AD,ADOSC_3_10,OBV_min_2,OBV_max_2,OBVe_4,OBVe_12,AOBV_LR_2,AOBV_SR_2,CMF_20,PVT
date,,,,,,,,,,,,,,,,,,,,,,,
2021-03-08,120.9375,121.0000,116.1875,116.3750,153918592.0,-1.799754e+08,-4.050747,23.851101,1371.392822,8.715573e+10,1310.421265,1.791228e+10,3.0,7.870897e+10,-9.719947e+07,8.715573e+10,8.730965e+10,8.724857e+10,8.742175e+10,0,1,-0.116858,2.983229e+10
2021-03-09,119.0000,122.0625,118.8125,121.0625,129159600.0,-6.777385e+07,-3.022866,30.521265,1375.420166,8.728489e+10,1310.421265,1.563638e+10,2.0,7.875864e+10,-8.281099e+07,8.715573e+10,8.728489e+10,8.726310e+10,8.740070e+10,0,1,-0.117862,3.035246e+10
2021-03-10,121.6875,122.1875,119.4375,120.0000,111760400.0,-7.505551e+07,-2.555383,30.390823,1374.542725,8.717313e+10,1310.421265,1.341125e+10,4.0,7.869260e+10,-9.041049e+07,8.717313e+10,8.728489e+10,8.722711e+10,8.736569e+10,0,1,-0.115577,3.025440e+10
2021-03-11,122.5625,123.1875,121.2500,121.9375,102753600.0,-3.589256e+07,-2.524725,30.751091,1376.156982,8.727588e+10,1310.421265,1.252952e+10,2.0,7.865945e+10,-9.584633e+07,8.717313e+10,8.727588e+10,8.724662e+10,8.735187e+10,0,1,-0.119369,3.042027e+10
2021-03-12,120.3750,121.1875,119.1875,121.0000,87963400.0,-4.254587e+07,-2.274888,31.111531,1375.387939,8.718791e+10,1310.421265,1.064357e+10,4.0,7.873092e+10,-6.661727e+07,8.718791e+10,8.727588e+10,8.722314e+10,8.732665e+10,0,1,-0.090416,3.035262e+10


In [45]:
dftest.shape

(5333, 23)

In [2]:
tickerDF = pd.read_csv('../01_data/SnP500_constituents.csv')#Google Drive/Code_for_bothComputers/PythonStockMarketEvalTool/PythonStockScreener/01_data/SnP500_constituents.csv

In [5]:
tickerDF.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ABMD,Abiomed,Health Care


In [9]:
tickerDF.Sector.unique().shape

(11,)

In [25]:
masterDf.FILEPATH.iloc[2]

'C:\\Users\\phili\\Google Drive\\Code_for_bothComputers\\PythonStockMarketEvalTool\\PythonStockScreener\\01_data\\01_price_data\\AAPL1'

In [31]:
pd.read_feather('C:\\Users\\phili\\Google Drive\\Code_for_bothComputers\\PythonStockMarketEvalTool\\PythonStockScreener\\01_data\\01_price_data\\AAPL1')

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1999-12-31,0.776367,0.791016,0.765137,0.790527,163811200.0,0.0,0.0
1,2000-01-03,0.806641,0.865234,0.782227,0.860840,535796800.0,0.0,0.0
2,2000-01-04,0.832520,0.850586,0.778320,0.788086,512377600.0,0.0,0.0
3,2000-01-05,0.797852,0.850098,0.791992,0.799805,778321600.0,0.0,0.0
4,2000-01-06,0.816406,0.822754,0.730469,0.730469,767972800.0,0.0,0.0
...,...,...,...,...,...,...,...,...
5332,2021-03-12,120.375000,121.187500,119.187500,121.000000,87963400.0,0.0,0.0
5333,2021-03-15,121.437500,124.000000,120.437500,124.000000,92403800.0,0.0,0.0
5334,2021-03-16,125.687500,127.250000,124.750000,125.562500,114740000.0,0.0,0.0
5335,2021-03-17,124.062500,125.875000,122.312500,124.750000,111437504.0,0.0,0.0
